In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import plotly.express as px
from plotly.offline import init_notebook_mode

In [146]:
from pymarkowitz import *

In [4]:
init_notebook_mode(connected=True)

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
sp500 = pd.read_csv("datasets/sp500_1990_2000.csv", index_col='DATE').drop(["Unnamed: 0"], axis=1)
sp500.head()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
DATE,,,,,,,,,,,,,,,,,,,,,
01/02/1990,1.330,5.875,3.874,1.282,8.4918,5.0313,33.125,1.1750,148.9705,6.1875,...,10.6667,2.8281,32.250,4.9662,5.9219,11.6129,12.5000,1.0417,25.4128,3.5000
01/03/1990,1.339,5.813,3.888,1.345,8.5382,5.1563,33.000,1.1500,148.7940,6.1563,...,10.4583,2.7031,32.375,5.0755,5.8906,11.3571,12.3750,1.0417,25.5775,3.5625
01/04/1990,1.344,5.625,3.881,1.408,8.4918,5.0313,32.500,1.1250,146.6759,6.1563,...,10.4167,2.6719,32.750,4.9194,5.8594,11.2548,12.2500,1.0000,25.6324,3.5625
01/05/1990,1.348,5.750,3.839,1.440,8.0741,5.0313,31.750,1.1250,142.2633,6.1563,...,10.2500,2.5938,32.000,4.9350,5.7969,11.2036,12.2188,1.0625,25.6873,3.5625
01/08/1990,1.357,5.750,3.839,1.456,8.2134,4.9844,31.625,1.1333,141.3807,6.1563,...,10.1667,2.6719,32.250,4.9974,5.8750,11.2036,12.3750,1.0208,25.5226,3.5625


## Calculating Comoment Matrix
- In this demo I will go through how to use the package to calculate comoment matrices (covariance, coskewness, cokurtosis) and beta. 

#### Moment
- Moment is a quantitative measure in statistics. Zeroth moment being total probability, 1st Moment is Expected Value, 2nd Moment is Variance, 3rd Moment is Skewness, 4th moment is Kurtosis and etc.
- The nth central moment is defined as the E(X - mean of X) ** n
- For a portfolio with many different assets, its nth moment is defined as E(X - mean of X)E(Y - mean of Y)E(Z - mean of Z).....

#### Classic Comoment Matrix
- Covariance Matrix (Moment = 2) Defined as E(X - mean of X)E(Y - mean of Y) for all pairs of X,Y in the portfolio
- Covariance matrix can also be normalized by dividing each value in the matrix by the standard deviation of X,Y underlying. A normalized covariance matrix is also known as the correlation matrix

- Higher moment comoment matrices are often normalized. For example a coskewness/cokurtosis matrix is always used after normalization

In [7]:
### Initialize a Return Generator
ret_generator = ReturnGenerator(sp500)
sp_return = ret_generator.calc_return(method='daily')

In [8]:
### Initialize a Moment Generator
mom_generator = MomentGenerator(sp_return)

### Features of Covariance


#### Calculation Techniques
 - The most common technique to calculate covariance matrix is sample covariance.
 - Other techniques are available in the sklearn.covariance module that can be used to construct covariance matrices
 
#### Sample Covariance
 - MomentGenerator allows for different weights based on observation's proximity to present
 - Exponential Weight (Exponential Decay)
 - Equal Weight
 - Customized Weight (Pass in a weight array)
   
#### Semivariance
 - In portfolio optimizations, sometimes we focus on the downside covariance as assets tend to move in the same direction during selloff periods.
 - Therefore, semivariance can be calculated by ignoring all returns above a certain threshold
 - Sometimes, daily returns are assumed to have a mean of 0 so you can also customize whether to ignore all returns and set them equal to 0, or set them equal to the average of returns below the threshold(which will not affect variance).

### Sklearn Techniques
- Note that some of the techniques have high run time with large number of observations/assets

In [15]:
### Annualized
mom_generator.calc_cov_mat("ShrunkCovariance", time_scaling=252)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,0.275317,0.011339,0.014546,0.081366,0.009320,0.055095,0.003992,0.017054,0.017067,0.005126,...,0.003860,0.021170,0.015358,0.021174,0.027712,0.013433,0.006397,0.009271,0.030683,0.010651
ABMD,0.011339,0.672230,0.009376,0.034354,0.007271,0.006800,-0.001279,0.005947,0.008382,0.006434,...,0.003399,0.004812,0.014126,0.007192,0.010537,0.006161,0.002267,0.007781,0.003953,0.013409
ABT,0.014546,0.009376,0.083345,0.020743,0.016068,0.014245,0.010845,0.020385,0.021973,0.006840,...,0.006983,0.021724,0.012786,0.009992,0.026851,0.014967,0.011633,0.009803,0.012109,0.009170
ADBE,0.081366,0.034354,0.020743,0.318003,0.017688,0.074535,0.009024,0.025449,0.023662,0.014095,...,0.008086,0.028640,0.024759,0.024918,0.029939,0.020653,0.008061,0.019047,0.035308,0.014863
ADM,0.009320,0.007271,0.016068,0.017688,0.085659,0.012514,0.007578,0.013299,0.015277,0.003205,...,0.005989,0.017268,0.009928,0.010756,0.017987,0.014516,0.010699,0.003599,0.007922,0.007697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.013433,0.006161,0.014967,0.020653,0.014516,0.012071,0.007853,0.013699,0.017877,0.006115,...,0.005312,0.021110,0.023964,0.014687,0.020380,0.100722,0.009868,0.010572,0.011857,0.011445
XOM,0.006397,0.002267,0.011633,0.008061,0.010699,0.010627,0.008322,0.007536,0.011728,0.002901,...,0.005815,0.011717,0.009490,0.012489,0.014061,0.009868,0.055768,0.003876,0.007867,0.005151
XRAY,0.009271,0.007781,0.009803,0.019047,0.003599,0.014213,0.003774,0.009308,0.010491,0.005598,...,0.004786,0.009927,0.010388,0.005953,0.008819,0.010572,0.003876,0.131602,0.013260,0.007828
XRX,0.030683,0.003953,0.012109,0.035308,0.007922,0.026163,0.004527,0.016270,0.018829,0.008945,...,0.002312,0.020190,0.015272,0.016628,0.018154,0.011857,0.007867,0.013260,0.140342,0.011573


In [17]:
### You can also construct the semivariance matrix with these techniques
#### In this case I am calculating the covariance when individualreturns are below 1%
mom_generator.calc_cov_mat("ShrunkCovariance", time_scaling=252, semi=True, bm_return=-0.01, assume_zero=False)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,0.054203,0.002361,0.000725,0.005019,0.000194,0.001919,0.000452,0.000830,0.001103,0.000514,...,-0.000050,0.000552,0.000375,0.001096,0.001106,0.000699,0.000069,-0.000049,0.000674,-0.000068
ABMD,0.002361,0.084259,0.000706,0.001763,0.000625,0.001806,0.000518,-0.000307,0.000322,-0.000229,...,0.000195,0.000018,0.000767,0.000435,0.000340,-0.000514,0.000775,-0.000063,-0.000276,-0.000256
ABT,0.000725,0.000706,0.008425,0.001799,0.000816,0.000973,0.000283,0.000886,0.001045,0.000147,...,0.000214,0.000682,0.000510,0.001060,0.001247,0.001058,0.000606,0.000840,0.000802,0.000818
ADBE,0.005019,0.001763,0.001799,0.048333,0.000985,0.004136,0.000569,0.001076,0.001724,0.000961,...,0.000312,0.001469,0.001042,0.002442,0.001438,0.000767,0.000668,0.001317,0.001351,0.001162
ADM,0.000194,0.000625,0.000816,0.000985,0.008648,0.000840,0.000214,0.000651,0.000866,0.000088,...,0.000196,0.000704,0.000583,0.001024,0.001228,0.000934,0.000444,0.000639,0.000663,0.000572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.000699,-0.000514,0.001058,0.000767,0.000934,0.000872,0.000280,0.000731,0.000983,0.000155,...,0.000280,0.000790,0.001046,0.000629,0.000918,0.008897,0.000492,0.000406,0.000717,0.001087
XOM,0.000069,0.000775,0.000606,0.000668,0.000444,0.000803,0.000253,0.000456,0.000470,0.000115,...,0.000307,0.000397,0.000516,0.000328,0.000713,0.000492,0.004287,0.000169,0.000448,0.000167
XRAY,-0.000049,-0.000063,0.000840,0.001317,0.000639,0.000770,0.000017,0.000376,0.000847,0.000025,...,0.000309,0.000682,0.000471,0.000763,0.000602,0.000406,0.000169,0.017392,0.000929,-0.000062
XRX,0.000674,-0.000276,0.000802,0.001351,0.000663,0.001858,0.000125,0.000585,0.001352,0.000161,...,0.000041,0.001291,0.001018,0.001326,0.001499,0.000717,0.000448,0.000929,0.037298,0.000481


In [18]:
### You can also normalize the covariance matrix to get the correlation matrix
mom_generator.calc_cov_mat("ShrunkCovariance", time_scaling=252, semi=True, bm_return=-0.01, assume_zero=False, normalize=True)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,1.000000,0.034934,0.033936,0.098059,0.008962,0.040156,0.032985,0.035144,0.058739,0.028651,...,-0.003476,0.025823,0.015243,0.041244,0.044425,0.031817,0.004503,-0.001607,0.014998,-0.002353
ABMD,0.034934,1.000000,0.026486,0.027621,0.023170,0.030310,0.030340,-0.010427,0.013772,-0.010243,...,0.010879,0.000681,0.025003,0.013134,0.010947,-0.018778,0.040781,-0.001645,-0.004930,-0.007085
ABT,0.033936,0.026486,1.000000,0.089149,0.095651,0.051647,0.052329,0.095161,0.141100,0.020799,...,0.037758,0.080906,0.052510,0.101191,0.126976,0.122245,0.100890,0.069384,0.045222,0.071502
ADBE,0.098059,0.027621,0.089149,1.000000,0.048160,0.091660,0.043963,0.048280,0.097197,0.056693,...,0.022990,0.072740,0.044813,0.097354,0.061143,0.036981,0.046426,0.045433,0.031810,0.042409
ADM,0.008962,0.023170,0.095651,0.048160,1.000000,0.044027,0.039127,0.068992,0.115478,0.012305,...,0.034123,0.082392,0.059278,0.096546,0.123508,0.106479,0.072945,0.052074,0.036901,0.049298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.031817,-0.018778,0.122245,0.036981,0.106479,0.045060,0.050360,0.076395,0.129231,0.021319,...,0.048209,0.091104,0.104902,0.058457,0.090977,1.000000,0.079670,0.032616,0.039357,0.092477
XOM,0.004503,0.040781,0.100890,0.046426,0.072945,0.059780,0.065649,0.068744,0.088988,0.022836,...,0.076038,0.065955,0.074516,0.043901,0.101857,0.079670,1.000000,0.019609,0.035413,0.020410
XRAY,-0.001607,-0.001645,0.069384,0.045433,0.052074,0.028433,0.002251,0.028090,0.079629,0.002494,...,0.037984,0.056249,0.033753,0.050714,0.042703,0.032616,0.019609,1.000000,0.036492,-0.003783
XRX,0.014998,-0.004930,0.045222,0.031810,0.036901,0.046865,0.010959,0.029880,0.086783,0.010791,...,0.003418,0.072745,0.049860,0.060175,0.072548,0.039357,0.035413,0.036492,1.000000,0.019964


#### Sample Covariance
 - The most common type of covariance matrix is constructed with equal weight
 - All parameters are set to default (assuming annualizing daily returns)

In [20]:
mom_generator.calc_cov_mat()

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,0.292015,0.012603,0.016168,0.090439,0.010359,0.061238,0.004437,0.018955,0.018969,0.005698,...,0.004291,0.023531,0.017071,0.023535,0.030801,0.014931,0.007110,0.010305,0.034104,0.011838
ABMD,0.012603,0.733184,0.010422,0.038184,0.008081,0.007558,-0.001422,0.006610,0.009317,0.007151,...,0.003778,0.005348,0.015701,0.007994,0.011711,0.006848,0.002520,0.008649,0.004393,0.014904
ABT,0.016168,0.010422,0.078639,0.023056,0.017859,0.015833,0.012054,0.022658,0.024423,0.007603,...,0.007762,0.024146,0.014212,0.011106,0.029845,0.016636,0.012931,0.010896,0.013459,0.010192
ADBE,0.090439,0.038184,0.023056,0.339461,0.019661,0.082846,0.010030,0.028286,0.026300,0.015666,...,0.008987,0.031833,0.027520,0.027696,0.033277,0.022956,0.008959,0.021171,0.039245,0.016520
ADM,0.010359,0.008081,0.017859,0.019661,0.081210,0.013910,0.008423,0.014781,0.016980,0.003563,...,0.006657,0.019194,0.011035,0.011955,0.019992,0.016135,0.011892,0.004000,0.008805,0.008555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.014931,0.006848,0.016636,0.022956,0.016135,0.013417,0.008728,0.015226,0.019870,0.006797,...,0.005905,0.023464,0.026636,0.016324,0.022652,0.097953,0.010968,0.011751,0.013179,0.012721
XOM,0.007110,0.002520,0.012931,0.008959,0.011892,0.011811,0.009250,0.008376,0.013035,0.003225,...,0.006463,0.013023,0.010548,0.013882,0.015629,0.010968,0.047987,0.004308,0.008744,0.005726
XRAY,0.010305,0.008649,0.010896,0.021171,0.004000,0.015798,0.004195,0.010345,0.011661,0.006222,...,0.005320,0.011034,0.011546,0.006616,0.009803,0.011751,0.004308,0.132276,0.014738,0.008701
XRX,0.034104,0.004393,0.013459,0.039245,0.008805,0.029081,0.005032,0.018084,0.020928,0.009943,...,0.002569,0.022442,0.016974,0.018482,0.020178,0.013179,0.008744,0.014738,0.141991,0.012863


#### Sample Covariance
- Exponential Weight can also be specified with two parameters: decay, span
- Example: decay=0.95, span=20 will convert to a weight of (0.95, ....., 0.95, 0.95^2, 0.95^2, 0.95^2, .........) with 20 values of 0.95, 20 values of 0.95^2 and etc...

** Note that the Generator assumes you pass in the data with most recent data at the bottom, so it will reverse the weight to give more weight towards later observations

In [23]:
mom_generator.calc_cov_mat(method='exp', decay=0.95, span=20)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,0.550169,0.048003,0.005536,0.174779,-0.006196,0.100154,0.004827,0.019682,0.018247,0.007683,...,-0.000665,0.036942,0.021054,0.042564,0.033309,0.002440,0.005628,0.008098,0.076946,0.036974
ABMD,0.048003,0.735058,-0.007773,0.105905,0.015853,0.049685,-0.029826,-0.010851,0.007724,0.004398,...,-0.014465,0.003284,0.020562,0.021940,0.013617,0.001328,-0.004393,0.029180,-0.021749,0.033418
ABT,0.005536,-0.007773,0.111522,0.016647,0.019812,-0.002408,0.020118,0.030169,0.040182,0.017840,...,0.008497,0.030157,0.011647,0.010877,0.038434,0.019907,0.016932,0.011486,0.008220,0.019277
ADBE,0.174779,0.105905,0.016647,0.506436,0.022813,0.119306,0.004328,0.038667,0.038725,0.044555,...,0.005022,0.041469,0.046300,0.058469,0.026808,0.016805,-0.001735,0.030614,0.073689,0.047178
ADM,-0.006196,0.015853,0.019812,0.022813,0.111122,0.004084,0.013825,0.022039,0.027554,0.008383,...,0.009900,0.028876,0.016574,0.016075,0.026447,0.022714,0.013464,0.002886,-0.004381,0.020435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.002440,0.001328,0.019907,0.016805,0.022714,-0.003183,0.010888,0.019351,0.035443,0.023483,...,0.010071,0.034473,0.055405,0.014918,0.034029,0.157145,0.009998,0.010413,-0.009487,0.022297
XOM,0.005628,-0.004393,0.016932,-0.001735,0.013464,-0.000559,0.017897,0.011923,0.018378,0.006094,...,0.012046,0.014752,0.014186,0.028033,0.015515,0.009998,0.079610,0.008937,0.012812,0.011326
XRAY,0.008098,0.029180,0.011486,0.030614,0.002886,0.022002,0.004530,0.014344,0.015739,0.016983,...,0.008380,0.014639,0.016106,0.010732,0.001998,0.010413,0.008937,0.127452,0.030231,0.016969
XRX,0.076946,-0.021749,0.008220,0.073689,-0.004381,0.060434,0.000697,0.026146,0.020350,0.019892,...,-0.008463,0.030356,0.015803,0.026198,0.013983,-0.009487,0.012812,0.030231,0.450628,0.020679


#### Sample Covariance
- Exponential Weight can also be flexibly combined with semivariance to look at downside only
- The generator is built with flexibility in mind to generate all possible combinations of methods, techniques with semivariance/normal variance and correlation matrices

In [26]:
mom_generator.calc_cov_mat(method='exp', semi=True, decay=0.95, span=20, bm_return=-0.01, assume_zero=False)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,0.184712,0.004355,0.004174,0.017451,-0.000175,0.005306,0.000772,0.000905,0.001582,0.000941,...,-0.000356,-0.002076,0.000593,0.003306,0.002325,0.002071,0.000039,-0.001099,-0.000886,-0.002576
ABMD,0.004355,0.083220,0.000534,0.008498,0.001099,0.003760,0.001404,0.001046,0.000978,-0.000695,...,-0.001160,0.000482,0.001898,0.004738,-0.000136,-0.002369,0.000711,-0.000001,-0.000626,0.000632
ABT,0.004174,0.000534,0.011350,0.008211,0.001142,0.001473,0.000441,0.001138,0.002422,0.000340,...,-0.000086,0.001621,0.000530,0.002903,0.002488,0.002399,0.001274,0.000874,0.002237,0.002187
ADBE,0.017451,0.008498,0.008211,0.083509,0.003879,0.009194,0.001886,0.002072,0.006141,0.003589,...,0.001183,0.004498,0.004557,0.007508,0.006511,0.003489,0.001987,0.002199,0.006836,0.001860
ADM,-0.000175,0.001099,0.001142,0.003879,0.008977,-0.000326,0.000377,0.000619,0.001940,0.000467,...,0.000119,0.001106,0.000363,0.001588,0.002817,0.002249,0.000403,0.000593,0.000509,0.001736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.002071,-0.002369,0.002399,0.003489,0.002249,0.001372,0.000636,0.001627,0.002999,0.000771,...,0.001058,0.002685,0.002284,0.001170,0.002514,0.014194,0.001038,0.000660,0.002705,0.003070
XOM,0.000039,0.000711,0.001274,0.001987,0.000403,0.000702,0.000669,0.000550,0.000927,0.000721,...,0.000870,0.000526,0.000872,0.001011,0.001052,0.001038,0.005794,0.000318,0.002277,0.000371
XRAY,-0.001099,-0.000001,0.000874,0.002199,0.000593,0.001932,-0.000257,0.000627,0.001305,0.001189,...,0.000221,0.000825,0.000414,0.001070,0.000812,0.000660,0.000318,0.018076,0.004345,-0.000382
XRX,-0.000886,-0.000626,0.002237,0.006836,0.000509,0.007320,-0.000014,-0.000462,0.004484,0.001058,...,0.000135,0.004546,0.004062,0.005091,0.006004,0.002705,0.002277,0.004345,0.180330,0.001940


In [27]:
mom_generator.calc_cov_mat(method='exp', semi=True, decay=0.95, span=20, bm_return=-0.01, assume_zero=False, normalize=True)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,1.000000,0.035125,0.091163,0.140511,-0.004292,0.050271,0.027119,0.019169,0.035095,0.022124,...,-0.010525,-0.042998,0.009478,0.047923,0.035311,0.040442,0.001200,-0.019020,-0.004857,-0.030128
ABMD,0.035125,1.000000,0.017375,0.101935,0.040194,0.053083,0.073532,0.033009,0.032330,-0.024364,...,-0.051174,0.014885,0.045218,0.102325,-0.003079,-0.068940,0.032366,-0.000032,-0.005107,0.011013
ABT,0.091163,0.017375,1.000000,0.266698,0.113113,0.056305,0.062460,0.097316,0.216715,0.032233,...,-0.010268,0.135424,0.034177,0.169756,0.152463,0.188998,0.157086,0.061035,0.049454,0.103223
ADBE,0.140511,0.101935,0.266698,1.000000,0.141661,0.129569,0.098554,0.065304,0.202601,0.125533,...,0.052082,0.138527,0.108345,0.161856,0.147100,0.101340,0.090333,0.056609,0.055703,0.032364
ADM,-0.004292,0.040194,0.113113,0.141661,1.000000,-0.013995,0.060028,0.059460,0.195174,0.049851,...,0.015928,0.103867,0.026328,0.104416,0.194115,0.199232,0.055863,0.046576,0.012646,0.092119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.040442,-0.068940,0.188998,0.101340,0.199232,0.046894,0.080571,0.124339,0.240017,0.065441,...,0.112942,0.200583,0.131745,0.061172,0.137760,1.000000,0.114459,0.041186,0.053471,0.129535
XOM,0.001200,0.032366,0.157086,0.090333,0.055863,0.037546,0.132693,0.065780,0.116091,0.095714,...,0.145506,0.061526,0.078677,0.082746,0.090218,0.114459,1.000000,0.031092,0.070446,0.024518
XRAY,-0.019020,-0.000032,0.061035,0.056609,0.046576,0.058530,-0.028860,0.042459,0.092569,0.089355,...,0.020906,0.054626,0.021170,0.049565,0.039414,0.041186,0.031092,1.000000,0.076102,-0.014286
XRX,-0.004857,-0.005107,0.049454,0.055703,0.012646,0.070194,-0.000513,-0.009903,0.100668,0.025181,...,0.004051,0.095266,0.065718,0.074686,0.092300,0.053471,0.070446,0.076102,1.000000,0.022963


#### Sample Covariance
- Customized Weight Array (Must be in the same length as the return data)
- Don't need to sum up to 1, as long as relative weights are provided
- This can also be combined with semivariance

In [31]:
# 2868 observations
customized_weight = np.random.uniform(low=0, high=10, size=sp_return.shape[0])

In [32]:
mom_generator.calc_cov_mat(method='custom', weights=customized_weight)

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG,...,WEC,WFC,WHR,WMB,WMT,WY,XOM,XRAY,XRX,ZION
AAPL,0.319153,0.004784,0.014949,0.093685,0.009910,0.062506,0.004311,0.015132,0.016637,0.005226,...,0.004374,0.023399,0.015196,0.028308,0.028566,0.013554,0.007540,0.010274,0.035487,0.009608
ABMD,0.004784,0.690762,0.008710,0.037829,0.012945,0.010237,-0.001548,0.003232,0.005434,0.011430,...,0.004263,0.006932,0.017844,0.008040,0.011747,0.001577,0.003403,0.009751,-0.000438,0.013429
ABT,0.014949,0.008710,0.078637,0.024568,0.016999,0.014581,0.012541,0.022126,0.024342,0.006655,...,0.009388,0.025332,0.013862,0.011316,0.031737,0.015973,0.013246,0.011062,0.010191,0.010022
ADBE,0.093685,0.037829,0.024568,0.328159,0.017840,0.084225,0.012280,0.026022,0.024389,0.016583,...,0.010120,0.033939,0.025779,0.029822,0.032760,0.023275,0.009071,0.023077,0.035625,0.017104
ADM,0.009910,0.012945,0.016999,0.017840,0.082853,0.015279,0.007747,0.013787,0.017699,0.004040,...,0.006998,0.019432,0.012328,0.011853,0.020731,0.015589,0.012554,0.004397,0.007204,0.006357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.013554,0.001577,0.015973,0.023275,0.015589,0.013855,0.009557,0.014781,0.020399,0.008236,...,0.006816,0.023418,0.024777,0.016590,0.022241,0.098907,0.010702,0.012583,0.011115,0.010787
XOM,0.007540,0.003403,0.013246,0.009071,0.012554,0.012132,0.009762,0.008343,0.013414,0.003966,...,0.006556,0.012526,0.011541,0.014311,0.015795,0.010702,0.049518,0.005250,0.008165,0.004216
XRAY,0.010274,0.009751,0.011062,0.023077,0.004397,0.016826,0.004578,0.008351,0.010998,0.007005,...,0.005660,0.011067,0.010995,0.006368,0.009635,0.012583,0.005250,0.132939,0.015154,0.006334
XRX,0.035487,-0.000438,0.010191,0.035625,0.007204,0.029024,0.005680,0.016401,0.019718,0.010207,...,0.003230,0.022204,0.015217,0.018489,0.015429,0.011115,0.008165,0.015154,0.154731,0.010652


#### Coskewness Matrix
- Coskewness matrix calculates the lopsidedness of the distribution. For portfolios, it refers to seeing more negative/positive returns.
- Coskewness matrix has a dimension of (N, N^2) N is the number of assets
- Coskewness can also be calculated with customized weights/semivariance settings
- However, as this matrix is not as often used as covariance matrix, the only technique to calculate is sample coskewness

** Note that coskewness matrix calculation may not work when data size is too large so in this demo I will reduce the sample size to 10 stocks, 100 observations each

In [64]:
mom_generator = MomentGenerator(sp_return.iloc[:100, :10])

#### Coskewness Matrix
- Coskewness is default to be normalized (it does not have a time unit and cannot be scaled, just like the correlation matrix)
- In the example below, the first cell which has an index of AAPL and a column of (AAPL, AAPL) is the skewness of AAPL. You can compare with the pandas builtin function below as they yield identical results.
- In fact you can check that the skewness values are identical for all results

In [65]:
mom_generator.calc_coskew_mat()

AAPL                                                              \
          AAPL      ABMD       ABT      ADBE       ADM      ADSK       AEP   
AAPL  0.428603  0.091426  0.235908  0.149080  0.180822  0.265404  0.340649   
ABMD  0.091426 -0.068651 -0.048122 -0.061065  0.051174 -0.033222 -0.111588   
ABT   0.235908 -0.048122 -0.011012 -0.078028  0.089773  0.165071  0.134587   
ADBE  0.149080 -0.061065 -0.078028  0.209558  0.038421  0.155296 -0.039850   
ADM   0.180822  0.051174  0.089773  0.038421 -0.082805  0.006103 -0.007486   
ADSK  0.265404 -0.033222  0.165071  0.155296  0.006103  0.304021  0.106425   
AEP   0.340649 -0.111588  0.134587 -0.039850 -0.007486  0.106425  0.107798   
AFL  -0.004601 -0.020007 -0.002456 -0.460281 -0.022825  0.066510  0.190760   
AIG   0.313818  0.069206  0.161521 -0.026693  0.067366  0.085176 -0.071756   
AJG  -0.130207  0.096497 -0.119069 -0.009540  0.011803 -0.113170 -0.211314   

                                    ...       AJG                      \
           AFL       AIG       AJG  ...      AAPL      ABMD       ABT   
AAPL -0.004601  0.313818 -0.130207  ... -0.130207  0.096497 -0.119069   
ABMD -0.020007  0.069206  0.096497  ...  0.096497  0.035266 -0.107996   
ABT  -0.002456  0.161521 -0.119069  ... -0.119069 -0.107996 -0.176547   
ADBE -0.460281 -0.026693 -0.009540  ... -0.009540 -0.138883 -0.036314   
ADM  -0.022825  0.067366  0.011803  ...  0.011803 -0.239569 -0.109327   
ADSK  0.066510  0.085176 -0.113170  ... -0.113170 -0.119998  0.037647   
AEP   0.190760 -0.071756 -0.211314  ... -0.211314 -0.120053 -0.079901   
AFL   0.214206  0.142572 -0.059108  ... -0.059108 -0.198996 -0.092395   
AIG   0.142572  0.110935 -0.085876  ... -0.085876  0.118236 -0.015084   
AJG  -0.059108 -0.085876  0.031298  ...  0.031298  0.184951  0.287663   

                                                                            
          ADBE       ADM      ADSK       AEP       AFL       AIG       AJG  
AAPL -0.009540  0.011803 -0.113170 -0.211314 -0.059108 -0.085876  0.031298  
ABMD -0.138883 -0.239569 -0.119998 -0.120053 -0.198996  0.118236  0.184951  
ABT  -0.036314 -0.109327  0.037647 -0.079901 -0.092395 -0.015084  0.287663  
ADBE  0.195993  0.016459  0.227674  0.044110  0.090890  0.001012  0.110043  
ADM   0.016459 -0.168838 -0.009677 -0.174014 -0.054858 -0.154899  0.145059  
ADSK  0.227674 -0.009677  0.172169  0.105237 -0.071698 -0.056502  0.146243  
AEP   0.044110 -0.174014  0.105237  0.162891 -0.142632 -0.220587 -0.242828  
AFL   0.090890 -0.054858 -0.071698 -0.142632 -0.294402 -0.098213  0.068306  
AIG   0.001012 -0.154899 -0.056502 -0.220587 -0.098213 -0.106121  0.250742  
AJG   0.110043  0.145059  0.146243 -0.242828  0.068306  0.250742  0.730788  

[10 rows x 100 columns]

In [67]:
sp_return.iloc[:100, :10].skew()

AAPL    0.428603
ABMD    1.102755
ABT    -0.373383
ADBE    0.426711
ADM     0.215474
ADSK    0.090873
AEP    -0.722740
AFL     0.743961
AIG    -0.083791
AJG     0.730788
dtype: float64

In [71]:
mom_generator.calc_coskew_mat().iloc[np.arange(0, 10), np.arange(0, 10) * 11]

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG
,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG
AAPL,0.428603,-0.068651,-0.011012,0.209558,-0.082805,0.304021,0.107798,0.214206,0.110935,0.031298
ABMD,0.091426,1.102755,-0.418079,-0.297950,0.148071,-0.165082,0.212444,-0.003383,0.374123,0.184951
ABT,0.235908,-0.101504,-0.373383,0.085758,0.014177,-0.050855,-0.081685,0.316140,-0.139601,0.287663
ADBE,0.149080,-0.222961,-0.163823,0.426711,-0.111230,0.300258,-0.012473,-0.163155,-0.179935,0.110043
ADM,0.180822,0.130631,-0.072314,0.144181,0.215474,0.080327,-0.336768,0.331763,-0.114460,0.145059
ADSK,0.265404,-0.088480,-0.019932,0.303228,-0.014368,0.090873,-0.186226,0.051075,-0.176392,0.146243
AEP,0.340649,-0.092613,-0.176084,0.095621,-0.209705,-0.057692,-0.722740,0.645725,-0.333824,-0.242828
AFL,-0.004601,-0.012930,0.048484,-0.039068,0.224131,-0.173886,0.043501,0.743961,0.044430,0.068306
AIG,0.313818,0.229556,-0.087387,0.043743,-0.023739,-0.136259,-0.027219,0.332236,-0.083791,0.250742


#### Coskewness Matrix
- You can also calculate coskewness with an exponential decay, semivariance, or customized weight just the same as calculating covariance matrix

In [73]:
mom_generator.calc_coskew_mat(method='exp', decay=0.95, span=10)

AAPL                                                              \
          AAPL      ABMD       ABT      ADBE       ADM      ADSK       AEP   
AAPL  0.454552  0.108191  0.243806  0.157547  0.217184  0.266849  0.298836   
ABMD  0.108191 -0.085099 -0.048310 -0.056277  0.058984 -0.043071 -0.127294   
ABT   0.243806 -0.048310  0.026804 -0.053598  0.124519  0.170325  0.130431   
ADBE  0.157547 -0.056277 -0.053598  0.200634  0.045474  0.160531 -0.029798   
ADM   0.217184  0.058984  0.124519  0.045474 -0.075981  0.012631  0.002642   
ADSK  0.266849 -0.043071  0.170325  0.160531  0.012631  0.330240  0.119274   
AEP   0.298836 -0.127294  0.130431 -0.029798  0.002642  0.119274  0.129887   
AFL  -0.041819  0.001061  0.014598 -0.416022  0.025398  0.046117  0.183013   
AIG   0.329726  0.058823  0.177200 -0.021707  0.086695  0.099841 -0.061008   
AJG  -0.098666  0.101313 -0.105286 -0.024084  0.030714 -0.126367 -0.221718   

                                    ...       AJG                      \
           AFL       AIG       AJG  ...      AAPL      ABMD       ABT   
AAPL -0.041819  0.329726 -0.098666  ... -0.098666  0.101313 -0.105286   
ABMD  0.001061  0.058823  0.101313  ...  0.101313  0.049579 -0.088232   
ABT   0.014598  0.177200 -0.105286  ... -0.105286 -0.088232 -0.159752   
ADBE -0.416022 -0.021707 -0.024084  ... -0.024084 -0.143159 -0.037956   
ADM   0.025398  0.086695  0.030714  ...  0.030714 -0.242782 -0.093634   
ADSK  0.046117  0.099841 -0.126367  ... -0.126367 -0.141771  0.018596   
AEP   0.183013 -0.061008 -0.221718  ... -0.221718 -0.101717 -0.078932   
AFL   0.188398  0.152702 -0.052859  ... -0.052859 -0.206848 -0.092707   
AIG   0.152702  0.101848 -0.095967  ... -0.095967  0.129838 -0.012236   
AJG  -0.052859 -0.095967  0.038927  ...  0.038927  0.202292  0.305058   

                                                                            
          ADBE       ADM      ADSK       AEP       AFL       AIG       AJG  
AAPL -0.024084  0.030714 -0.126367 -0.221718 -0.052859 -0.095967  0.038927  
ABMD -0.143159 -0.242782 -0.141771 -0.101717 -0.206848  0.129838  0.202292  
ABT  -0.037956 -0.093634  0.018596 -0.078932 -0.092707 -0.012236  0.305058  
ADBE  0.207066  0.025958  0.236195  0.037319  0.085738 -0.004574  0.097446  
ADM   0.025958 -0.144762 -0.026427 -0.174602 -0.059386 -0.136084  0.197619  
ADSK  0.236195 -0.026427  0.199214  0.100715 -0.062014 -0.058513  0.131520  
AEP   0.037319 -0.174602  0.100715  0.198034 -0.132515 -0.198312 -0.278826  
AFL   0.085738 -0.059386 -0.062014 -0.132515 -0.248279 -0.089526  0.083854  
AIG  -0.004574 -0.136084 -0.058513 -0.198312 -0.089526 -0.110149  0.228383  
AJG   0.097446  0.197619  0.131520 -0.278826  0.083854  0.228383  0.728876  

[10 rows x 100 columns]

In [74]:
mom_generator.calc_coskew_mat(method='custom', weights=np.random.randn(100))

AAPL                                                              \
           AAPL      ABMD       ABT      ADBE       ADM      ADSK       AEP   
AAPL  13.571815  0.048088  6.087349  4.071377  2.457252  7.812256  4.556620   
ABMD   0.048088 -0.741398  1.195960  1.111979 -0.454995  1.105165  0.160143   
ABT    6.087349  1.195960  4.083989  2.463843  0.749022  3.359737  0.504514   
ADBE   4.071377  1.111979  2.463843  1.189777  0.239267 -0.021976 -2.031885   
ADM    2.457252 -0.454995  0.749022  0.239267  0.364539  1.480509  1.230211   
ADSK   7.812256  1.105165  3.359737 -0.021976  1.480509  2.099042 -1.269761   
AEP    4.556620  0.160143  0.504514 -2.031885  1.230211 -1.269761 -2.591541   
AFL    0.838263  0.602717  0.694949 -0.365436  0.519207  0.718609  0.826829   
AIG    3.210510 -1.148776 -0.083239 -0.253651 -0.235296  1.084632 -0.429426   
AJG   -1.909305 -1.641798 -0.437807  0.952135 -1.478145  1.334797  2.063518   

                                    ...       AJG                      \
           AFL       AIG       AJG  ...      AAPL      ABMD       ABT   
AAPL  0.838263  3.210510 -1.909305  ... -1.909305 -1.641798 -0.437807   
ABMD  0.602717 -1.148776 -1.641798  ... -1.641798 -1.828625 -3.401808   
ABT   0.694949 -0.083239 -0.437807  ... -0.437807 -3.401808 -2.134227   
ADBE -0.365436 -0.253651  0.952135  ...  0.952135 -1.321934 -0.789989   
ADM   0.519207 -0.235296 -1.478145  ... -1.478145 -1.772793 -0.192587   
ADSK  0.718609  1.084632  1.334797  ...  1.334797  0.768956  1.888895   
AEP   0.826829 -0.429426  2.063518  ...  2.063518 -0.154725  0.728179   
AFL   0.461567  0.296041 -0.449938  ... -0.449938 -0.963858  0.034288   
AIG   0.296041  1.603065  0.273232  ...  0.273232 -1.526638  0.327304   
AJG  -0.449938  0.273232 -2.344239  ... -2.344239 -3.993895 -2.885399   

                                                                            
          ADBE       ADM      ADSK       AEP       AFL       AIG       AJG  
AAPL  0.952135 -1.478145  1.334797  2.063518 -0.449938  0.273232 -2.344239  
ABMD -1.321934 -1.772793  0.768956 -0.154725 -0.963858 -1.526638 -3.993895  
ABT  -0.789989 -0.192587  1.888895  0.728179  0.034288  0.327304 -2.885399  
ADBE -2.353631  0.047270 -0.950429 -1.544376  0.438843  0.222721 -2.764698  
ADM   0.047270  2.546350  1.988685  1.277014 -0.305488 -1.009487 -4.335263  
ADSK -0.950429  1.988685 -1.774700 -2.756866  0.165749 -0.238802  0.135857  
AEP  -1.544376  1.277014 -2.756866 -3.576780 -1.617377 -0.846581  4.080612  
AFL   0.438843 -0.305488  0.165749 -1.617377 -0.041099 -1.231123  0.315397  
AIG   0.222721 -1.009487 -0.238802 -0.846581 -1.231123  1.691889  2.243073  
AJG  -2.764698 -4.335263  0.135857  4.080612  0.315397  2.243073 -5.947669  

[10 rows x 100 columns]

In [78]:
mom_generator.calc_coskew_mat(method='custom', weights=np.random.dirichlet(np.arange(1, 101)), semi=True, bm_return=-0.02)

AAPL                                                          \
              AAPL          ABMD           ABT          ADBE           ADM   
AAPL -4.448697e+00  7.078136e-01  3.693367e-02 -5.563693e-01  4.461316e-01   
ABMD  7.078136e-01 -6.399553e-02  4.324827e-02  1.908653e-01 -1.002426e-03   
ABT   3.693367e-02  4.324827e-02  2.453532e-01  2.746381e-15 -5.686897e-03   
ADBE -5.563693e-01  1.908653e-01  2.746381e-15  1.578499e+00  4.167957e-16   
ADM   4.461316e-01 -1.002426e-03 -5.686897e-03  4.167957e-16  4.401807e-01   
ADSK  1.893509e+00 -2.515468e-01  4.917690e-03 -2.131430e-01 -1.139842e-04   
AEP  -5.243117e-14  5.681112e-15 -1.401708e-15  1.092803e-14 -6.805625e-15   
AFL   2.870424e-01  3.165944e-02 -3.642585e-17  1.681679e-01  2.658171e-01   
AIG  -2.866431e-01  2.174122e-15  6.671193e-16  4.023359e-15 -2.098666e-01   
AJG   5.976978e-01 -1.298170e-14  1.078708e-15 -2.469108e-14  1.571488e-14   

                                                                            \
              ADSK           AEP           AFL           AIG           AJG   
AAPL  1.893509e+00 -5.243117e-14  2.870424e-01 -2.866431e-01  5.976978e-01   
ABMD -2.515468e-01  5.681112e-15  3.165944e-02  2.174122e-15 -1.298170e-14   
ABT   4.917690e-03 -1.401708e-15 -3.642585e-17  6.671193e-16  1.078708e-15   
ADBE -2.131430e-01  1.092803e-14  1.681679e-01  4.023359e-15 -2.469108e-14   
ADM  -1.139842e-04 -6.805625e-15  2.658171e-01 -2.098666e-01  1.571488e-14   
ADSK -4.179201e-01  7.573032e-15 -7.575197e-16  3.001287e-15 -1.748690e-14   
AEP   7.573032e-15  6.438114e-29 -7.255271e-16  5.021910e-15 -8.863792e-15   
AFL  -7.575197e-16 -7.255271e-16  5.531446e-02 -7.898700e-16  2.561948e-15   
AIG   3.001287e-15  5.021910e-15 -7.898700e-16  3.889943e-01  3.850942e-02   
AJG  -1.748690e-14 -8.863792e-15  2.561948e-15  3.850942e-02  2.185486e+00   

      ...           AJG                                            \
      ...          AAPL          ABMD           ABT          ADBE   
AAPL  ...  5.976978e-01 -1.298170e-14  1.078708e-15 -2.469108e-14   
ABMD  ... -1.298170e-14  6.522456e-14  1.915687e-14  7.879977e-15   
ABT   ...  1.078708e-15  1.915687e-14  3.645461e-14 -9.594072e-16   
ADBE  ... -2.469108e-14  7.879977e-15 -9.594072e-16  8.951596e-14   
ADM   ...  1.571488e-14 -3.097110e-15 -2.595444e-15 -3.521394e-16   
ADSK  ... -1.748690e-14  9.457736e-15  4.309622e-16 -1.853953e-14   
AEP   ... -8.863792e-15 -8.565068e-29 -2.161295e-29  2.810847e-28   
AFL   ...  2.561948e-15 -6.593053e-15  1.517583e-29  5.883728e-15   
AIG   ...  3.850942e-02 -2.552950e-15 -2.226985e-16  7.495406e-15   
AJG   ...  2.185486e+00  1.012573e-15 -1.600041e-14  4.024934e-15   

                                                                            \
               ADM          ADSK           AEP           AFL           AIG   
AAPL  1.571488e-14 -1.748690e-14 -8.863792e-15  2.561948e-15  3.850942e-02   
ABMD -3.097110e-15  9.457736e-15 -8.565068e-29 -6.593053e-15 -2.552950e-15   
ABT  -2.595444e-15  4.309622e-16 -2.161295e-29  1.517583e-29 -2.226985e-16   
ADBE -3.521394e-16 -1.853953e-14  2.810847e-28  5.883728e-15  7.495406e-15   
ADM   5.076382e-14 -9.989020e-18 -3.132240e-15  9.492111e-15 -5.248393e-15   
ADSK -9.989020e-18  1.073597e-13 -9.385611e-15 -1.477895e-14 -2.372292e-14   
AEP  -3.132240e-15 -9.385611e-15  3.080716e-14  3.307354e-29 -3.794174e-15   
AFL   9.492111e-15 -1.477895e-14  3.307354e-29  5.476086e-14  4.213467e-17   
AIG  -5.248393e-15 -2.372292e-14 -3.794174e-15  4.213467e-17 -1.682774e-02   
AJG   1.079282e-27  5.973077e-29 -6.572092e-14  6.573971e-15 -3.701534e-02   

                    
               AJG  
AAPL  2.185486e+00  
ABMD  1.012573e-15  
ABT  -1.600041e-14  
ADBE  4.024934e-15  
ADM   1.079282e-27  
ADSK  5.973077e-29  
AEP  -6.572092e-14  
AFL   6.573971e-15  
AIG  -3.701534e-02  
AJG   1.793670e+00  

[10 rows x 100 columns]

#### Cokurtosis Matrix
- Cokurtosis matrix calculates the heaviness of the tailof the distribution. For portfolios, it refers to seeing large moves in one direction of all individual constituents at the same time
- COkurtosis matrix has a dimension of (N, N^3) N is the number of assets
- Cokurtosis can also be calculated with customized weights/semivariance settings just like coskewness or cokurtosis
- You can compare the kurtosis values calculated with pandas builtin function versus the generator through the diagonal

In [86]:
mom_generator.calc_cokurt_mat() - 3 # -3 to get excess kurtosis

AAPL                                                              \
          AAPL                                                               
          AAPL      ABMD       ABT      ADBE       ADM      ADSK       AEP   
AAPL  2.442505 -2.525912 -1.107368 -1.743894 -2.170246 -1.325820 -2.722348   
ABMD -2.525912 -2.439770 -2.710005 -2.773373 -2.820649 -2.875212 -3.070662   
ABT  -1.107368 -2.710005 -1.636952 -2.387739 -2.424796 -2.202830 -2.647618   
ADBE -1.743894 -2.773373 -2.387739 -1.620882 -2.703295 -2.190074 -2.850641   
ADM  -2.170246 -2.820649 -2.424796 -2.703295 -2.205288 -2.687151 -2.900854   
ADSK -1.325820 -2.875212 -2.202830 -2.190074 -2.687151 -1.672333 -2.502580   
AEP  -2.722348 -3.070662 -2.647618 -2.850641 -2.900854 -2.502580 -1.875698   
AFL  -2.440061 -2.826708 -2.345988 -3.236116 -2.831032 -2.917255 -2.684068   
AIG  -2.152949 -2.904404 -2.507044 -2.822122 -2.700768 -2.571522 -2.643888   
AJG  -3.046120 -2.915968 -2.967244 -2.887367 -2.915296 -3.085179 -3.360499   

                                    ...       AJG                      \
                                    ...       AJG                       
           AFL       AIG       AJG  ...      AAPL      ABMD       ABT   
AAPL -2.440061 -2.152949 -3.046120  ... -2.300835 -2.989142 -2.850797   
ABMD -2.826708 -2.904404 -2.915968  ... -2.989142 -1.861255 -2.589657   
ABT  -2.345988 -2.507044 -2.967244  ... -2.850797 -2.589657 -2.270548   
ADBE -3.236116 -2.822122 -2.887367  ... -2.912803 -3.152179 -2.545155   
ADM  -2.831032 -2.700768 -2.915296  ... -2.965370 -2.773618 -2.776938   
ADSK -2.917255 -2.571522 -3.085179  ... -3.056549 -3.237659 -2.707222   
AEP  -2.684068 -2.643888 -3.360499  ... -3.164486 -3.071614 -2.953825   
AFL  -1.229216 -2.729555 -3.100697  ... -2.878826 -2.805058 -2.755829   
AIG  -2.729555 -2.308082 -2.981411  ... -2.849565 -2.735956 -2.522091   
AJG  -3.100697 -2.981411 -2.300835  ... -2.770703 -3.073693 -2.251771   

                                                                            
                                                                            
          ADBE       ADM      ADSK       AEP       AFL       AIG       AJG  
AAPL -2.912803 -2.965370 -3.056549 -3.164486 -2.878826 -2.849565 -2.770703  
ABMD -3.152179 -2.773618 -3.237659 -3.071614 -2.805058 -2.735956 -3.073693  
ABT  -2.545155 -2.776938 -2.707222 -2.953825 -2.755829 -2.522091 -2.251771  
ADBE -1.633535 -3.127270 -2.093830 -2.826176 -2.838350 -2.596709 -1.780583  
ADM  -3.127270 -1.895414 -3.130764 -3.051004 -2.959487 -3.137692 -3.502858  
ADSK -2.093830 -3.130764 -1.952631 -2.771576 -2.725481 -2.662074 -2.131338  
AEP  -2.826176 -3.051004 -2.771576 -2.163249 -2.909710 -2.916089 -3.474661  
AFL  -2.838350 -2.959487 -2.725481 -2.909710 -2.419557 -2.751993 -2.750411  
AIG  -2.596709 -3.137692 -2.662074 -2.916089 -2.751993 -1.776032 -1.700771  
AJG  -1.780583 -3.502858 -2.131338 -3.474661 -2.750411 -1.700771  1.611767  

[10 rows x 1000 columns]

In [89]:
mom_generator.calc_cokurt_mat().iloc[np.arange(0, 10), np.arange(0, 10) * 111] - 3

,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG
,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG
,AAPL,ABMD,ABT,ADBE,ADM,ADSK,AEP,AFL,AIG,AJG
AAPL,2.442505,-2.889004,-2.227200,-2.057829,-2.826427,-2.280383,-3.264220,-1.628677,-2.483120,-2.770703
ABMD,-2.525912,4.134607,-1.579015,-3.255142,-2.708690,-3.098547,-3.334900,-2.512392,-2.267660,-3.073693
ABT,-1.107368,-1.041104,0.342793,-1.725998,-1.729064,-1.940712,-1.647925,-1.317670,-1.671134,-2.251771
ADBE,-1.743894,-3.120458,-1.731595,1.080028,-3.179115,-1.847756,-2.754127,-3.200568,-2.691606,-1.780583
ADM,-2.170246,-1.774101,-1.570008,-3.218735,0.290623,-2.965425,-1.612173,-2.279900,-1.518785,-3.502858
ADSK,-1.325820,-3.889505,-2.187663,-1.171186,-2.815139,-0.090765,-1.927312,-2.222956,-2.442042,-2.131338
AEP,-2.722348,-2.606461,-2.216097,-2.258853,-1.682160,-2.775785,1.040657,-1.507039,-1.882685,-3.474661
AFL,-2.440061,-2.467010,-1.655702,-2.636201,-1.877847,-2.452975,-1.960300,1.245902,-2.154618,-2.750411


In [97]:
sp_return.iloc[:100, :10].kurt()

AAPL    2.403831
ABMD    4.112854
ABT     0.283122
ADBE    1.027729
ADM     0.230430
ADSK   -0.154771
AEP     0.987964
AFL     1.195262
AIG     0.198592
AJG     1.564785
dtype: float64

### Higher Comoment Matrix
- While the current usage of higher comoment matrices in portfolio construction is rare, you can still construct them with the flexibility of weights and semivariance option that are available in all lower comoment calculations
- With a moment of X, the dimension of the output will be (N, N^(X-1))

In [90]:
mom_generator.calc_comoment_mat(5)

AAPL                                                              \
          AAPL                                                               
          AAPL                                                               
          AAPL      ABMD       ABT      ADBE       ADM      ADSK       AEP   
AAPL  8.618836  0.815095  2.861020  3.452803  1.297805  3.781868  2.983467   
ABMD  0.815095 -0.095750 -0.003500  0.150630  0.093663  0.195161 -0.039648   
ABT   2.861020 -0.003500  0.603374  0.673322  0.220417  1.224948  1.009115   
ADBE  3.452803  0.150630  0.673322  2.193182  0.423493  1.813268  0.358066   
ADM   1.297805  0.093663  0.220417  0.423493 -0.109125  0.384607 -0.079871   
ADSK  3.781868  0.195161  1.224948  1.813268  0.384607  1.896337  1.020555   
AEP   2.983467 -0.039648  1.009115  0.358066 -0.079871  1.020555  0.799672   
AFL  -1.367149 -0.400357 -0.605480 -2.372618 -0.532091 -0.608063  0.717768   
AIG   2.757592  0.289234  1.016732  0.558367  0.338359  0.950647  0.378282   
AJG  -1.605459  0.097281 -0.790439 -0.393455 -0.130084 -0.837057 -0.960356   

                                    ...       AJG                      \
                                    ...       AJG                       
                                    ...       AJG                       
           AFL       AIG       AJG  ...      AAPL      ABMD       ABT   
AAPL -1.367149  2.757592 -1.605459  ...  0.230131  0.576442  0.011644   
ABMD -0.400357  0.289234  0.097281  ...  0.576442  0.394484  0.343888   
ABT  -0.605480  1.016732 -0.790439  ...  0.011644  0.343888  0.647618   
ADBE -2.372618  0.558367 -0.393455  ... -0.214071  0.091601  0.914579   
ADM  -0.532091  0.338359 -0.130084  ...  0.304467 -1.132908 -0.472231   
ADSK -0.608063  0.950647 -0.837057  ... -0.474647 -0.228464  0.715927   
AEP   0.717768  0.378282 -0.960356  ... -0.587901 -0.116976 -0.157642   
AFL   1.607981  0.320252 -0.397990  ... -0.038050 -0.233102  0.080045   
AIG   0.320252  0.586121 -0.387655  ... -0.234469  1.140987  1.083367   
AJG  -0.397990 -0.387655  0.513146  ... -0.025267  1.658684  2.894318   

                                                                            
                                                                            
                                                                            
          ADBE       ADM      ADSK       AEP       AFL       AIG       AJG  
AAPL -0.214071  0.304467 -0.474647 -0.587901 -0.038050 -0.234469 -0.025267  
ABMD  0.091601 -1.132908 -0.228464 -0.116976 -0.233102  1.140987  1.658684  
ABT   0.914579 -0.472231  0.715927 -0.157642  0.080045  1.083367  2.894318  
ADBE  2.066629  0.101746  1.842113  0.207838  0.494376  0.973838  2.937090  
ADM   0.101746  0.043938 -0.074773 -0.448251 -0.404117 -0.548713  0.516652  
ADSK  1.842113 -0.074773  1.590452  0.419458  0.177335  0.672182  2.470181  
AEP   0.207838 -0.448251  0.419458  0.898892 -0.045577 -0.331557 -1.322352  
AFL   0.494376 -0.404117  0.177335 -0.045577 -0.300678  0.274949  1.022930  
AIG   0.973838 -0.548713  0.672182 -0.331557  0.274949  1.040594  2.867212  
AJG   2.937090  0.516652  2.470181 -1.322352  1.022930  2.867212  9.302873  

[10 rows x 10000 columns]

### Beta
- Calculating Beta is essentially calculating the covariance matrix combined with the benchmark
- Therefore all parameters available in calculating covariance matrix are available in calculating beta
- You can customize with customized weight arrays/exponential decay weights/equal weights
- You can also capture downside beta by using a semivariance matrix
- You can also capture special beta values using sklearn.covariance functions

In [98]:
mom_generator = MomentGenerator(sp_return)

#### Price-Average as Beta Vector
- I will construct a benchmark by summing all stock prices rowwise
- Below are some customizable ways to capture beta

In [103]:
mom_generator.calc_beta(beta_vec=sp_return.sum(axis=1)/sp_return.shape[1])

AAPL    1.354177
ABMD    0.853478
ABT     0.996289
ADBE    1.841079
ADM     0.797473
          ...   
WY      1.029585
XOM     0.622185
XRAY    0.662008
XRX     1.073552
ZION    0.805197
Length: 203, dtype: float64

In [104]:
mom_generator.calc_beta(beta_vec=sp_return.sum(axis=1)/sp_return.shape[1], technique='OAS')

AAPL    1.231559
ABMD    0.776197
ABT     0.906077
ADBE    1.674373
ADM     0.725264
          ...   
WY      0.936358
XOM     0.565848
XRAY    0.602065
XRX     0.976344
ZION    0.732288
Length: 203, dtype: float64

In [108]:
### Downside Beta
mom_generator.calc_beta(beta_vec=sp_return.sum(axis=1)/sp_return.shape[1], 
                        technique='sample', method='default', semi=True, bm_return=-0.02)

AAPL    0.850545
ABMD    0.717199
ABT     0.767535
ADBE    1.061101
ADM     0.670448
          ...   
WY      0.635054
XOM     0.513406
XRAY    0.549222
XRX     0.819518
ZION    0.697716
Length: 203, dtype: float64

In [109]:
mom_generator.calc_beta(beta_vec=sp_return.sum(axis=1)/sp_return.shape[1], 
                        technique='sample', method='exp', decay=0.95, span=100)

AAPL    1.331101
ABMD    0.739664
ABT     0.922439
ADBE    1.776748
ADM     0.742105
          ...   
WY      0.969408
XOM     0.613003
XRAY    0.608474
XRX     1.070351
ZION    0.920891
Length: 203, dtype: float64

In [119]:
mom_generator.calc_beta(beta_vec=sp_return.sum(axis=1)/sp_return.shape[1], 
                        technique='sample', method='custom', weights=np.random.dirichlet(np.arange(1,1001)))

AAPL    1.724191
ABMD    1.115674
ABT     1.177186
ADBE    2.216399
ADM     0.959792
          ...   
WY      1.387098
XOM     0.451994
XRAY    0.921003
XRX     1.041795
ZION    0.608300
Length: 203, dtype: float64

### Plot some stocks' "beta"

In [144]:
selected_beta = mom_generator.calc_beta(beta_vec=sp_return.sum(axis=1)/sp_return.shape[1]).iloc[np.random.randint(1, 200, 20)]

In [145]:
px.bar(pd.DataFrame(selected_beta, columns=['BETA']).reset_index().drop_duplicates(subset='index'), x='index', y='BETA')